# Módulo 12: Modelos  de Regressão
# Exercício 1: Regressão Múltipla II

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [5]:
# carregando bibliotecas

import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings

%matplotlib inline

In [6]:
# Desativar avisos de FutureWarning específicos

warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

In [7]:
# carregando arquivo e construindo DataFrame

arquivo = pd.read_csv('PREVISAO_DE_RENDA.csv')
df_previsao = pd.DataFrame(arquivo)

In [8]:
# informações sobre os tipos dos dados

df_previsao.dtypes

Unnamed: 0                 int64
data_ref                  object
id_cliente                 int64
sexo                      object
posse_de_veiculo            bool
posse_de_imovel             bool
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                      int64
tempo_emprego            float64
qt_pessoas_residencia    float64
renda                    float64
dtype: object

In [9]:
# informações sobre os dados

df_previsao.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [10]:
# informações sobre o número de linhas e colunas do dataFrame

print(f'O número de "linhas" do DataFrame é:   {df_previsao.shape[0]}')
print(f'O número de "colunas" do DataFrame é:   {df_previsao.shape[1]}')

O número de "linhas" do DataFrame é:   15000
O número de "colunas" do DataFrame é:   15


In [11]:
# verificando dados faltantes

df_previsao.isna().sum()

Unnamed: 0                  0
data_ref                    0
id_cliente                  0
sexo                        0
posse_de_veiculo            0
posse_de_imovel             0
qtd_filhos                  0
tipo_renda                  0
educacao                    0
estado_civil                0
tipo_residencia             0
idade                       0
tempo_emprego            2573
qt_pessoas_residencia       0
renda                       0
dtype: int64

In [12]:
# observando a porcentagem de dados faltantes

porcentagem_dados_faltantes = (df_previsao.isnull().sum() / len(df_previsao)) * 100
porcentagem_dados_faltantes

Unnamed: 0                0.000000
data_ref                  0.000000
id_cliente                0.000000
sexo                      0.000000
posse_de_veiculo          0.000000
posse_de_imovel           0.000000
qtd_filhos                0.000000
tipo_renda                0.000000
educacao                  0.000000
estado_civil              0.000000
tipo_residencia           0.000000
idade                     0.000000
tempo_emprego            17.153333
qt_pessoas_residencia     0.000000
renda                     0.000000
dtype: float64

In [13]:
# dropar todas as linhas de "tempo_emprego" que tenha NA

df_previsao.dropna(subset=['tempo_emprego'], inplace=True)

In [14]:
# informações sobre o número de linhas e colunas do dataFrame depois de dropar

print(f'O número de "linhas" do DataFrame é:   {df_previsao.shape[0]}')
print(f'O número de "colunas" do DataFrame é:   {df_previsao.shape[1]}')

O número de "linhas" do DataFrame é:   12427
O número de "colunas" do DataFrame é:   15


1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [15]:
# transformando os dados em 'dummies' usando o patsy

formula = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + ' \
          'tipo_renda + educacao + idade + educacao + tipo_residencia + ' \
          'tempo_emprego + qt_pessoas_residencia'

y, X = patsy.dmatrices(formula, df_previsao)

In [16]:
# regressão múltipla usando "statsmodels.api" - em função de 'log(renda)'

reg = sm.OLS(y,X).fit()
reg.summary()


# resumo:
# resíduo do y = 'renda'
# R-quadrado = 0.357
# 'P>|t|' maior que 5% = variável não é estatisticamente significante
# vamos tirar a variável 'posse_de_veiculo' pois tem 'P>|t| = 0.672' que é maior que 5%
# vamos tirar a variável 'qtd_filhos' pois tem 'P>|t| = 0.513' que é maior que 5%
# vamos tirar a variável 'qt_pessoas_residencia' pois tem 'P>|t| = 0.092' que é maior que 5%



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     343.9
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        17:27:44   Log-Likelihood:                -13576.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12406   BIC:                         2.735e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            7.1586      0.097     73.530      0.000       6.968       7.349
sexo[T.M]                            0.7849      0.015     53.738      0.000       0.756       0.813
posse_de_veiculo[T.True]             0.0449      0.014      3.180      0.001       0.017       0.073
posse_de_imovel[T.True]              0.0827      0.014      5.922      0.000       0.055       0.110
tipo_renda[T.Bolsista]               0.2183      0.241      0.905      0.366      -0.255       0.691
tipo_renda[T.Empresário]             0.1528      0.015     10.235      0.000       0.123       0.182
tipo_renda[T.Pensionista]           -0.3260      0.241     -1.352      0.176      -0.799       0.147
tipo_renda[T.Servidor público]       0.0578      0.022      2.599      0.009       0.014       0.101
educacao[T.Pós graduação]            0.0947      0.159      0.596      0.551      -0.217       0.406
educacao[T.Secundário]              -0.0206      0.072     -0.286      0.775      -0.162       0.120
educacao[T.Superior completo]        0.0872      0.072      1.207      0.228      -0.054       0.229
educacao[T.Superior incompleto]     -0.0519      0.078     -0.667      0.505      -0.204       0.100
tipo_residencia[T.Casa]             -0.0403      0.054     -0.744      0.457      -0.146       0.066
tipo_residencia[T.Com os pais]      -0.0226      0.060     -0.375      0.708      -0.141       0.096
tipo_residencia[T.Comunitário]      -0.1125      0.105     -1.067      0.286      -0.319       0.094
tipo_residencia[T.Estúdio]           0.0679      0.099      0.684      0.494      -0.127       0.263
tipo_residencia[T.Governamental]    -0.0512      0.066     -0.775      0.438      -0.181       0.078
qtd_filhos                           0.0304      0.019      1.570      0.116      -0.008       0.068
idade                                0.0058      0.001      7.315      0.000       0.004       0.007
tempo_emprego                        0.0616      0.001     59.304      0.000       0.060       0.064
qt_pessoas_residencia               -0.0023      0.016     -0.143      0.886      -0.034       0.029
==============================================================================
Omnibus:                        0.768   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.681   Jarque-Bera (JB):                0.742
Skew:                           0.017   Prob(JB):                        0.690
Kurtosis:                       3.018   Cond. No.                     1.59e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.59e+03. This might indicate that there are
stron

In [17]:
# transformando os dados em 'dummies' usando o patsy  - sem a variável 'posse_de_veiculo'
#                                                     - sem a variável 'qtd_filhos'
#                                                     - sem a variável 'qt_pessoas_residencia'

formula = 'np.log(renda) ~ sexo + posse_de_imovel + tipo_renda + ' \
          'educacao + idade + educacao + tipo_residencia + ' \
          'tempo_emprego'

y, X = patsy.dmatrices(formula, df_previsao)

In [18]:
# ajustando um modelo para prever log(renda) considerando todas as covariáveis disponíveis.

reg = sm.OLS(y,X).fit()
reg.summary()


# resumo
# AIC = quanto menor melhor
# R-quadrado ajustado = é o mesmo que o R-quadrado mas que penaliza pelo número de parâmetros


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     402.7
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        17:28:07   Log-Likelihood:                -13586.
No. Observations:               12427   AIC:                         2.721e+04
Df Residuals:                   12409   BIC:                         2.734e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            7.1988      0.093     77.007      0.000       7.016       7.382
sexo[T.M]                            0.8019      0.014     58.240      0.000       0.775       0.829
posse_de_imovel[T.True]              0.0854      0.014      6.116      0.000       0.058       0.113
tipo_renda[T.Bolsista]               0.1946      0.241      0.806      0.420      -0.278       0.668
tipo_renda[T.Empresário]             0.1504      0.015     10.091      0.000       0.121       0.180
tipo_renda[T.Pensionista]           -0.3284      0.241     -1.362      0.173      -0.801       0.144
tipo_renda[T.Servidor público]       0.0578      0.022      2.599      0.009       0.014       0.101
educacao[T.Pós graduação]            0.1197      0.159      0.753      0.451      -0.192       0.431
educacao[T.Secundário]              -0.0146      0.072     -0.202      0.840      -0.156       0.127
educacao[T.Superior completo]        0.0970      0.072      1.342      0.180      -0.045       0.239
educacao[T.Superior incompleto]     -0.0487      0.078     -0.626      0.531      -0.201       0.104
tipo_residencia[T.Casa]             -0.0326      0.054     -0.602      0.547      -0.139       0.073
tipo_residencia[T.Com os pais]      -0.0178      0.060     -0.294      0.769      -0.136       0.101
tipo_residencia[T.Comunitário]      -0.1012      0.105     -0.960      0.337      -0.308       0.105
tipo_residencia[T.Estúdio]           0.0847      0.099      0.852      0.394      -0.110       0.279
tipo_residencia[T.Governamental]    -0.0448      0.066     -0.679      0.497      -0.174       0.085
idade                                0.0049      0.001      6.517      0.000       0.003       0.006
tempo_emprego                        0.0616      0.001     59.332      0.000       0.060       0.064
==============================================================================
Omnibus:                        0.691   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.708   Jarque-Bera (JB):                0.671
Skew:                           0.017   Prob(JB):                        0.715
Kurtosis:                       3.013   Cond. No.                     1.59e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.59e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [54]:
#  VERIFICANDO COVARIÁVEIS
# 'log(renda)' e 'tipo_renda'

reg = smf.ols('np.log(renda) ~ C(tipo_renda) + sexo + posse_de_imovel + idade + tempo_emprego', data = df_previsao).fit()

reg.summary()

# resumo
# # AIC = quanto menor melhor = 2.727e+04
# R-quadrado = 0.352
# R-quadrado ajustado = é o mesmo que o R-quadrado mas que penaliza pelo número de parâmetros = 0.351
# quanto maior melhor

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     841.8
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        18:26:18   Log-Likelihood:                -13624.
No. Observations:               12427   AIC:                         2.727e+04
Df Residuals:                   12418   BIC:                         2.733e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             7.2089      0.032    227.180      0.000       7.147       7.271
C(tipo_renda)[T.Bolsista]             0.2724      0.242      1.126      0.260      -0.202       0.746
C(tipo_renda)[T.Empresário]           0.1614      0.015     10.861      0.000       0.132       0.190
C(tipo_renda)[T.Pensionista]         -0.2688      0.242     -1.112      0.266      -0.743       0.205
C(tipo_renda)[T.Servidor público]     0.0778      0.022      3.510      0.000       0.034       0.121
sexo[T.M]                             0.7991      0.014     58.106      0.000       0.772       0.826
posse_de_imovel[T.True]               0.0881      0.014      6.420      0.000       0.061       0.115
idade                                 0.0044      0.001      5.963      0.000       0.003       0.006
tempo_emprego                         0.0617      0.001     59.335      0.000       0.060       0.064
==============================================================================
Omnibus:                        0.973   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.615   Jarque-Bera (JB):                0.945
Skew:                           0.019   Prob(JB):                        0.623
Kurtosis:                       3.020   Cond. No.                     1.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [55]:
# verificando os valores da variável 'tipo_renda'

contagem_tipo_renda = df_previsao['tipo_renda'].value_counts()
contagem_tipo_renda

# resumo
# o valor mais frequente é "Assalariado" e já se encontra na casela de referência

tipo_renda
Assalariado         7633
Empresário          3508
Servidor público    1268
Bolsista               9
Pensionista            9
Name: count, dtype: int64

In [58]:
#  VERIFICANDO COVARIÁVEIS
# 'log(renda)' e 'educacao'

reg = smf.ols('np.log(renda) ~ C(educacao) + sexo + posse_de_imovel + idade + tempo_emprego', data = df_previsao).fit()

reg.summary()

# resumo
# AIC = quanto menor melhor = 2.730e+04
# R-quadrado = 0.350
# R-quadrado ajustado = é o mesmo que o R-quadrado mas que penaliza pelo número de parâmetros = 0.349
# quanto maior melhor

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.350
Model:                            OLS   Adj. R-squared:                  0.349
Method:                 Least Squares   F-statistic:                     835.3
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        18:28:56   Log-Likelihood:                -13641.
No. Observations:               12427   AIC:                         2.730e+04
Df Residuals:                   12418   BIC:                         2.737e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              7.2052      0.078     92.899      0.000       7.053       7.357
C(educacao)[T.Pós graduação]           0.1729      0.159      1.085      0.278      -0.139       0.485
C(educacao)[T.Secundário]              0.0056      0.072      0.077      0.939      -0.136       0.147
C(educacao)[T.Superior completo]       0.1296      0.072      1.793      0.073      -0.012       0.271
C(educacao)[T.Superior incompleto]    -0.0184      0.078     -0.236      0.813      -0.171       0.134
sexo[T.M]                              0.7956      0.014     57.842      0.000       0.769       0.823
posse_de_imovel[T.True]                0.0860      0.014      6.252      0.000       0.059       0.113
idade                                  0.0048      0.001      6.318      0.000       0.003       0.006
tempo_emprego                          0.0609      0.001     59.180      0.000       0.059       0.063
==============================================================================
Omnibus:                        0.852   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.653   Jarque-Bera (JB):                0.846
Skew:                           0.020   Prob(JB):                        0.655
Kurtosis:                       3.003   Cond. No.                     1.20e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [59]:
# verificando os valores da variável 'educacao'

contagem_educacao = df_previsao['educacao'].value_counts()
contagem_educacao

# resumo
# o valor mais frequente é "Secundário" e vamos colocá-lo na casela de referência

educacao
Secundário             7045
Superior completo      4695
Superior incompleto     558
Primário                103
Pós graduação            26
Name: count, dtype: int64

In [65]:
# 'log(renda)' e 'educacao'
#  mudando a casela de referência para "Secundário" que é a casela "2"

reg = smf.ols('np.log(renda) ~ C(educacao, Treatment(2)) + sexo + posse_de_imovel + idade + tempo_emprego', data = df_previsao).fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.350
Model:                            OLS   Adj. R-squared:                  0.349
Method:                 Least Squares   F-statistic:                     835.3
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        18:31:15   Log-Likelihood:                -13641.
No. Observations:               12427   AIC:                         2.730e+04
Df Residuals:                   12418   BIC:                         2.737e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.2108      0.033    219.187      0.000       7.146       7.275
C(educacao, Treatment(2))[T.Primário]               -0.0056      0.072     -0.077      0.939      -0.147       0.136
C(educacao, Treatment(2))[T.Pós graduação]           0.1673      0.143      1.173      0.241      -0.112       0.447
C(educacao, Treatment(2))[T.Superior completo]       0.1241      0.014      9.001      0.000       0.097       0.151
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0239      0.032     -0.740      0.459      -0.087       0.039
sexo[T.M]                                            0.7956      0.014     57.842      0.000       0.769       0.823
posse_de_imovel[T.True]                              0.0860      0.014      6.252      0.000       0.059       0.113
idade                                                0.0048      0.001      6.318      0.000       0.003       0.006
tempo_emprego                                        0.0609      0.001     59.180      0.000       0.059       0.063
==============================================================================
Omnibus:                        0.852   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.653   Jarque-Bera (JB):                0.846
Skew:                           0.020   Prob(JB):                        0.655
Kurtosis:                       3.003   Cond. No.                         933.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [67]:
#  VERIFICANDO COVARIÁVEIS
# 'log(renda)' e 'tipo_residencia'

reg = smf.ols('np.log(renda) ~ C(tipo_residencia) + sexo + posse_de_imovel + idade + tempo_emprego', data = df_previsao).fit()

reg.summary()

# resumo
# AIC = quanto menor melhor = 2.739e+04
# R-quadrado = 0.345
# R-quadrado ajustado = é o mesmo que o R-quadrado mas que penaliza pelo número de parâmetros = 0.345
# quanto maior melhor

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.345
Method:                 Least Squares   F-statistic:                     728.2
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        18:31:57   Log-Likelihood:                -13683.
No. Observations:               12427   AIC:                         2.739e+04
Df Residuals:                   12417   BIC:                         2.746e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               7.3041      0.061    119.623      0.000       7.184       7.424
C(tipo_residencia)[T.Casa]             -0.0257      0.054     -0.472      0.637      -0.132       0.081
C(tipo_residencia)[T.Com os pais]      -0.0226      0.061     -0.371      0.711      -0.142       0.097
C(tipo_residencia)[T.Comunitário]      -0.0774      0.106     -0.731      0.465      -0.285       0.130
C(tipo_residencia)[T.Estúdio]           0.1352      0.100      1.353      0.176      -0.061       0.331
C(tipo_residencia)[T.Governamental]    -0.0464      0.066     -0.699      0.484      -0.177       0.084
sexo[T.M]                               0.7876      0.014     56.981      0.000       0.761       0.815
posse_de_imovel[T.True]                 0.0896      0.014      6.373      0.000       0.062       0.117
idade                                   0.0042      0.001      5.611      0.000       0.003       0.006
tempo_emprego                           0.0610      0.001     59.002      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.166   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.558   Jarque-Bera (JB):                1.145
Skew:                           0.023   Prob(JB):                        0.564
Kurtosis:                       3.013   Cond. No.                         947.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [68]:
# verificando os valores da variável 'tipo_residencia'

contagem_tipo_residencia = df_previsao['tipo_residencia'].value_counts()
contagem_tipo_residencia

# resumo
# o valor mais frequente é "Casa" e vamos colocá-lo na casela de referência

tipo_residencia
Casa             11071
Com os pais        674
Governamental      360
Aluguel            183
Estúdio             75
Comunitário         64
Name: count, dtype: int64

In [69]:
# 'log(renda)' e 'educacao'
#  mudando a casela de referência para "Casa" que é a casela "1"

reg = smf.ols('np.log(renda) ~ C(tipo_residencia, Treatment(1)) + sexo + posse_de_imovel ', data = df_previsao).fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.130
Method:                 Least Squares   F-statistic:                     265.4
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        18:32:25   Log-Likelihood:                -15450.
No. Observations:               12427   AIC:                         3.092e+04
Df Residuals:                   12419   BIC:                         3.098e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             7.9735      0.015    535.947      0.000       7.944       8.003
C(tipo_residencia, Treatment(1))[T.Aluguel]          -0.1250      0.063     -1.993      0.046      -0.248      -0.002
C(tipo_residencia, Treatment(1))[T.Com os pais]      -0.1406      0.034     -4.163      0.000      -0.207      -0.074
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.2110      0.105     -2.005      0.045      -0.417      -0.005
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.2822      0.097      2.899      0.004       0.091       0.473
C(tipo_residencia, Treatment(1))[T.Governamental]     0.0042      0.045      0.092      0.927      -0.085       0.093
sexo[T.M]                                             0.6669      0.016     42.433      0.000       0.636       0.698
posse_de_imovel[T.True]                               0.0901      0.016      5.574      0.000       0.058       0.122
==============================================================================
Omnibus:                      146.095   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              159.175
Skew:                           0.235   Prob(JB):                     2.73e-35
Kurtosis:                       3.295   Cond. No.                         18.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

2. Remova a variável menos significante e analise: Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    
    -  **Resposta**
    - Em linhas gerais a insersão e remoção de variáveis categorizadas não alterou significativamente os índices que mensuram o modelo; 



3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
     
    -  **Resposta**
    - O melhor modelo foi o que tirarmos as variáveis **p-value** menos significativas, ou seja, aquelas que são maiores que 5%.



    